In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from prepare import prep_titanic, prep_telco, train_val_test

In [2]:
accuracy = '(TP + TN) /	(TP + TN + FP + FN)'
recall = 'TP / (TP + FN)'
true_positive_rate = 'TP / (TP + FN)'
false_positive_rate = 'FP / (FP + TN)'
true_negative_rate = 'TN / (TN + FP)'
false_negative_rate = 'FN / (FN + TP)'
precision = 'TP / (TP + FP)'
f1_score = '2 * (precision * recall) / (precision + recall)'
support = 'TP + FN'

# Create a pandas DataFrame for the metrics
data = {
    'Metric': ['Accuracy', 'Recall', 'True positive Rate' ,'False Positive Rate', 'True Negative Rate', 'False Negative Rate', 'Precision', 'F1-Score', 'Support'],
    'Value': [accuracy, recall, true_positive_rate ,false_positive_rate, true_negative_rate, false_negative_rate, precision, f1_score, support]
}

metrics = pd.DataFrame(data)

In [3]:
data = {
    '': ['Actual Positive', 'Actual Negative'] ,
    'Predicted Positive': ['True Positive (TP)', 'False Positive (FP)'],
    'Predicted Negative': ['False Negative (FN)', 'True Negative (TN)'], 
}

rubric = pd.DataFrame(data)
rubric.set_index('', inplace=True)

# Decision Tree

1) What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

### Titanic

In [4]:
#loading the titanic dataframe

df = prep_titanic()
df.head()

,survived,sex,age,sibsp,parch,fare,class,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,Southampton,0
1,1,female,38.0,1,0,71.2833,First,Cherbourg,0
2,1,female,26.0,0,0,7.9250,Third,Southampton,1
3,1,female,35.0,1,0,53.1000,First,Southampton,0
4,0,male,35.0,0,0,8.0500,Third,Southampton,1


In [5]:
# splitting the titanic dataframe into training, validation, and test sets.

train, val, test = train_val_test(df, strat = 'survived')

In [6]:
train.head()

,survived,sex,age,sibsp,parch,fare,class,embark_town,alone
748,0,male,19.0,1,0,53.1000,First,Southampton,0
45,0,male,28.0,0,0,8.0500,Third,Southampton,1
28,1,female,28.0,0,0,7.8792,Third,Queenstown,1
633,0,male,28.0,0,0,0.0000,First,Southampton,1
403,0,male,28.0,1,0,15.8500,Third,Southampton,0


In [7]:
#encoding the sex, class and embark categorical columns in the train set.

train = pd.get_dummies(train, columns = ['sex', 'class', 'embark_town'])
train = train.drop(columns = 'sex_male')
train.head()

,survived,age,sibsp,parch,fare,alone,sex_female,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
748,0,19.0,1,0,53.1000,0,False,True,False,False,False,False,True
45,0,28.0,0,0,8.0500,1,False,False,False,True,False,False,True
28,1,28.0,0,0,7.8792,1,True,False,False,True,False,True,False
633,0,28.0,0,0,0.0000,1,False,True,False,False,False,False,True
403,0,28.0,1,0,15.8500,0,False,False,False,True,False,False,True


In [8]:
#encoding the sex, class and embark categorical columns in the val set

val = pd.get_dummies(val, columns = ['sex', 'class', 'embark_town'])
val = val.drop(columns = 'sex_male')
val.head()

,survived,age,sibsp,parch,fare,alone,sex_female,class_First,class_Second,class_Third,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
377,0,27.0,0,2,211.500,0,False,True,False,False,True,False,False
244,0,30.0,0,0,7.225,1,False,False,False,True,True,False,False
72,0,21.0,0,0,73.500,1,False,False,True,False,False,False,True
815,0,28.0,0,0,0.000,1,False,True,False,False,False,False,True
841,0,16.0,0,0,10.500,1,False,False,True,False,False,False,True


In [9]:
#making sure train and val all have the same number of columns

train.shape, val.shape 

((623, 13), (134, 13))

In [10]:
# creating an X & y version of train and val, where y is a series with just the target variable and X are all the features.#

X_train = train.drop(columns = ['survived'])
y_train = train.survived

X_val = val.drop(columns = ['survived'])
y_val = val.survived

In [11]:
# creating a baseline of the most occuring number in the survived column

y_train.mode()

0    0
Name: survived, dtype: int64

In [12]:
base_acc = (y_train == 0).mean()
base_acc

0.6163723916532905

baseline accuracy is 61%.

2) Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [13]:
#verifying that x and y train have the same amount of rows

X_train.shape, y_train.shape

((623, 12), (623,))

In [14]:
#creating a model and fitting it to our train set

clf = DecisionTreeClassifier(random_state = 42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [15]:
clf.feature_importances_

array([2.70919255e-01, 3.99517916e-02, 2.15028407e-02, 2.11394109e-01,
       1.85222747e-02, 3.22080709e-01, 2.80461230e-02, 9.16681448e-04,
       6.81605310e-02, 6.94754114e-03, 2.38698233e-04, 1.13194446e-02])

In [16]:
#making predictions on X_train with the .predict() method.

t_pred = clf.predict(X_train)
t_pred[:10]

array([0, 0, 1, 0, 0, 1, 1, 0, 0, 0])

In [17]:
#making a probability estimation on X_train with the .predict_proba() method

t_prob = clf.predict_proba(X_train)
t_prob[:10]

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [18]:
#evaluating the accuracy of the train model

clf.score(X_train, y_train)

0.9791332263242376

The accuracy on the train model is 98%

3) Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [19]:
#using the .score() method to predict the y_train

clf.score(X_train, y_train)

0.9791332263242376

In [20]:
#creating a crosstab for the val confusion matrix

v_pred = clf.predict(X_val)
val_ct = pd.crosstab(y_val, v_pred)
val_ct

col_0,0,1
survived,,
0,70,12
1,17,35


In [21]:
#creating a crosstab for the train confusion matrix

train_ct = pd.crosstab(y_train, t_pred)
train_ct

col_0,0,1
survived,,
0,383,1
1,12,227


#confusion matrix

accuracy = (383 + 225) / (383 + 225 + 14 + 1)  
ans=> 98

precision = 383 / (383 + 14)  
ans=> 96

recall = 383 / (383 + 1)  
ans=> 99

In [22]:
#classification report

print(classification_report(y_train, t_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       384
           1       1.00      0.95      0.97       239

    accuracy                           0.98       623
   macro avg       0.98      0.97      0.98       623
weighted avg       0.98      0.98      0.98       623



4) Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [23]:
#creating a crosstab for the metrics below

train_ct

col_0,0,1
survived,,
0,383,1
1,12,227


#### train

In [24]:
# accuracy

t_acc = (383 + 225) / (383 + 225 + 14 + 1)
t_acc

0.9759229534510433

In [25]:
#true positive rate

t_tpr = 383 / (383 + 1)
t_tpr

0.9973958333333334

In [26]:
#false positive rate

fpr = 14 / (14 + 225)
fpr

0.058577405857740586

In [27]:
#true negative rate

tnr = 225 / (225 + 383)
tnr

0.37006578947368424

In [28]:
#false negative rate

fnr = 1 / (1 + 14)
fnr

0.06666666666666667

In [29]:
#precision

pre = 383 / (383 + 14)
pre

0.964735516372796

In [30]:
#recall

rec = 383 / (383 + 1)
rec

0.9973958333333334

In [31]:
#F1 score

F1 = 2 * (pre * rec) / (pre + rec)
F1

0.9807938540332908

In [32]:
#support

383 + 1

384

5) Run through steps 2-4 using a different max_depth value.

In [33]:
seed = 42
train_acc = []
val_acc = []
depth = []

for i in range(2, 10):

    clf = DecisionTreeClassifier(max_depth = i, random_state = seed)

    clf.fit(X_train, y_train)

    train_acc.append(clf.score(X_train, y_train))

    val_acc.append(clf.score(X_val, y_val))

    depth.append(i)

In [34]:
pd.DataFrame({'max_depth' : depth, 'train_acc' : train_acc, 'val_acc' : val_acc})

,max_depth,train_acc,val_acc
0,2,0.807384,0.798507
1,3,0.829856,0.835821
2,4,0.845907,0.835821
3,5,0.863563,0.820896
4,6,0.879615,0.820896
5,7,0.894061,0.798507
6,8,0.913323,0.798507
7,9,0.922953,0.813433


6) Which model performs better on your in-sample data?

Recall model works best for the in-sample data!

7) Which model performs best on your out-of-sample data, the validate set?

In [35]:
val_ct

col_0,0,1
survived,,
0,70,12
1,17,35


#### validate

In [36]:
#accuracy

v_acc = (73 + 36) / (73 + 36 + 16 + 9)
v_acc

0.8134328358208955

In [37]:
#precision

v_pre = 73 / (73 + 16)
v_pre

0.8202247191011236

In [38]:
#recall

v_rec = 73 / (73 + 9)
v_rec

0.8902439024390244

Recall works best for the out of sample data!

# Telco

1) What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [39]:
telco_df = prep_telco()
telco_df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


In [40]:
train, val, test = train_val_test(telco_df, strat = 'churn')

In [41]:
#encoding the categorical columns in the train set.

train = pd.get_dummies(train, columns = ['gender', 'contract_type', 'internet_service_type', 'payment_type',
                                        'partner', 'dependents', 'phone_service', 'multiple_lines',
                                         'online_security', 'online_backup', 'device_protection', 'churn',
                                         'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing'],
                       drop_first = True)

train.columns = train.columns.str.replace(' ', '_')
train = train.rename(columns = {'churn_Yes' : 'churn'} )
train.head()

,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,contract_type_One_year,contract_type_Two_year,internet_service_type_Fiber_optic,payment_type_Credit_card_(automatic),payment_type_Electronic_check,...,device_protection_No_internet_service,device_protection_Yes,churn,tech_support_No_internet_service,tech_support_Yes,streaming_tv_No_internet_service,streaming_tv_Yes,streaming_movies_No_internet_service,streaming_movies_Yes,paperless_billing_Yes
5609,0,14,76.45,1117.55,True,False,False,True,False,True,...,False,True,False,False,False,False,False,False,False,False
2209,0,5,70.00,347.4,True,True,False,False,False,False,...,False,True,True,False,False,False,True,False,True,True
6919,0,35,75.20,2576.2,True,False,False,True,False,True,...,False,False,True,False,False,False,False,False,False,True
2284,0,58,86.10,4890.5,True,False,True,False,False,True,...,False,True,False,False,False,False,True,False,True,True
845,0,2,49.60,114.7,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [42]:
#encoding the categorical columns in the val set.

val = pd.get_dummies(val, columns = ['gender', 'contract_type', 'internet_service_type', 'payment_type',
                                        'partner', 'dependents', 'phone_service', 'multiple_lines',
                                         'online_security', 'online_backup', 'device_protection', 'churn',
                                         'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing'], 
                     drop_first = True)

val.columns = val.columns.str.replace(' ', '_')
val = val.rename(columns = {'churn_Yes' : 'churn'} )
val.head()

,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,contract_type_One_year,contract_type_Two_year,internet_service_type_Fiber_optic,payment_type_Credit_card_(automatic),payment_type_Electronic_check,...,device_protection_No_internet_service,device_protection_Yes,churn,tech_support_No_internet_service,tech_support_Yes,streaming_tv_No_internet_service,streaming_tv_Yes,streaming_movies_No_internet_service,streaming_movies_Yes,paperless_billing_Yes
6910,0,46,20.20,845.6,True,True,False,False,False,False,...,True,False,False,True,False,True,False,True,False,True
6044,0,40,106.00,4178.65,True,True,False,True,False,False,...,False,True,False,False,False,False,True,False,True,True
2153,0,53,25.55,1336.1,True,False,True,False,True,False,...,True,False,False,True,False,True,False,True,False,False
2089,0,70,40.05,2799.75,True,False,True,False,False,False,...,False,False,False,False,True,False,False,False,True,True
6393,0,3,96.60,291.9,False,False,False,True,False,True,...,False,False,True,False,False,False,True,False,True,True


In [51]:
# Convert boolean columns to numeric
bool_columns = ['gender_Male', 'contract_type_One_year', 'contract_type_Two_year', 'internet_service_type_Fiber_optic',
                'payment_type_Credit_card_(automatic)', 'payment_type_Electronic_check',
                'device_protection_No_internet_service', 'device_protection_Yes',
                'tech_support_No_internet_service', 'tech_support_Yes',
                'streaming_tv_No_internet_service', 'streaming_tv_Yes',
                'streaming_movies_No_internet_service', 'streaming_movies_Yes',
                'paperless_billing_Yes']

train[bool_columns] = train[bool_columns].astype(int)

# Now you can proceed with creating and fitting the model
train.head()

,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,contract_type_One_year,contract_type_Two_year,internet_service_type_Fiber_optic,payment_type_Credit_card_(automatic),payment_type_Electronic_check,...,device_protection_No_internet_service,device_protection_Yes,churn,tech_support_No_internet_service,tech_support_Yes,streaming_tv_No_internet_service,streaming_tv_Yes,streaming_movies_No_internet_service,streaming_movies_Yes,paperless_billing_Yes
5609,0,14,76.45,1117.55,1,0,0,1,0,1,...,0,1,False,0,0,0,0,0,0,0
2209,0,5,70.00,347.4,1,1,0,0,0,0,...,0,1,True,0,0,0,1,0,1,1
6919,0,35,75.20,2576.2,1,0,0,1,0,1,...,0,0,True,0,0,0,0,0,0,1
2284,0,58,86.10,4890.5,1,0,1,0,0,1,...,0,1,False,0,0,0,1,0,1,1
845,0,2,49.60,114.7,0,0,0,0,0,0,...,0,0,True,0,0,0,0,0,0,1


In [52]:
# Convert boolean columns to numeric
bool_columns = ['gender_Male', 'contract_type_One_year', 'contract_type_Two_year', 'internet_service_type_Fiber_optic',
                'payment_type_Credit_card_(automatic)', 'payment_type_Electronic_check',
                'device_protection_No_internet_service', 'device_protection_Yes',
                'tech_support_No_internet_service', 'tech_support_Yes',
                'streaming_tv_No_internet_service', 'streaming_tv_Yes',
                'streaming_movies_No_internet_service', 'streaming_movies_Yes',
                'paperless_billing_Yes']

val[bool_columns] = val[bool_columns].astype(int)

# Now you can proceed with creating and fitting the model
val.head()

,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,contract_type_One_year,contract_type_Two_year,internet_service_type_Fiber_optic,payment_type_Credit_card_(automatic),payment_type_Electronic_check,...,device_protection_No_internet_service,device_protection_Yes,churn,tech_support_No_internet_service,tech_support_Yes,streaming_tv_No_internet_service,streaming_tv_Yes,streaming_movies_No_internet_service,streaming_movies_Yes,paperless_billing_Yes
6910,0,46,20.20,845.6,1,1,0,0,0,0,...,1,0,False,1,0,1,0,1,0,1
6044,0,40,106.00,4178.65,1,1,0,1,0,0,...,0,1,False,0,0,0,1,0,1,1
2153,0,53,25.55,1336.1,1,0,1,0,1,0,...,1,0,False,1,0,1,0,1,0,0
2089,0,70,40.05,2799.75,1,0,1,0,0,0,...,0,0,False,0,1,0,0,0,1,1
6393,0,3,96.60,291.9,0,0,0,1,0,1,...,0,0,True,0,0,0,1,0,1,1


In [60]:
train.total_charges.dtypes

dtype('O')

In [53]:
#making sure train and val all have the same number of columns

train.shape, val.shape 

((4930, 30), (1056, 30))

In [54]:
# creating an X & y version of train and val, where y is a series with just the target variable and X are all the features.

X_train = train.drop(columns = ['churn'])
y_train = train.churn

X_val = val.drop(columns = ['churn'])
y_val = val.churn

In [55]:
# creating a baseline of the most occuring number in the survived column

y_train.mode()

0    False
Name: churn, dtype: bool

In [56]:
base_acc = (y_train == 0).mean()
base_acc

0.734685598377282

baseline accuracy is 73%.

2) Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [57]:
#verifying that x and y train have the same amount of rows

X_train.shape, y_train.shape

((4930, 29), (4930,))

In [61]:
#convert total charges from object to int

In [ ]:
train